# Reformat the SC18 Mira dataset CSV

In [1]:
%matplotlib inline

In [2]:
import datetime
import pytz
import pandas

In [3]:
orig_df = pandas.read_csv('alcf-tokio-results-2_14_17-2_15_18.csv.gz')
new_df = pandas.read_csv('mira-summaries_2017-02-14_2018-02-15.csv.gz')

In [4]:
print("Which columns are inconsistent?\n")
_tmp_list = list(new_df.columns)
for col in list(orig_df.columns):
    if col not in _tmp_list:
        print("%s in orig_df; not in new_df" % col)
        
_tmp_list = list(orig_df.columns)
for col in list(new_df.columns):
    if col not in _tmp_list:
        print("%s in new_df; not in orig_df" % col)

Which columns are inconsistent?

Unnamed: 0 in orig_df; not in new_df
index in orig_df; not in new_df


The original CSV had timestamps formatted as strings in the Chicago time zone

In [5]:
orig_df['_datetime_start'].head()

0    2017-02-14 22:40:35
1    2017-02-14 22:42:29
2    2017-02-14 22:43:56
3    2017-02-14 22:47:55
4    2017-02-14 22:50:26
Name: _datetime_start, dtype: object

The new CSV's epoch timestamps were made without awareness that the date strings in the original were measured in a different time zone than the system here.  This caused the timestamps to be off by a few hours.

In [6]:
new_df['_datetime_start'].head()

0    1487112035
1    1487112149
2    1487112236
3    1487112475
4    1487112626
Name: _datetime_start, dtype: int64

This is how we localize the string-based timestamps and convert them into true (UTC-based) epoch timestamps.

In [7]:
tz_chicago = pytz.timezone("America/Chicago")
for date_str in orig_df['_datetime_start'].head().values:
    datetime_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    datetime_obj_tz = tz_chicago.localize(datetime_obj)
    print(date_str, int(datetime_obj_tz.timestamp()))

2017-02-14 22:40:35 1487133635
2017-02-14 22:42:29 1487133749
2017-02-14 22:43:56 1487133836
2017-02-14 22:47:55 1487134075
2017-02-14 22:50:26 1487134226
